<a href="https://colab.research.google.com/github/Norod/my-colab-experiments/blob/master/Buntworthy_StyleGAN_ADA_blending_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network blending in StyleGAN-ADA

Swapping layers between two models in StyleGAN gives some interesting results. You need a base model and a second model which has been fine-tuned from the base.

In [ ]:
%cd /content/
!git clone https://github.com/justinpinkney/stylegan2.git

In [ ]:
%cd stylegan2

In [ ]:
%tensorflow_version 1.x
!pip install typer

Command line help for the blending function is below. Make sure that you specify either `--output-grid` to save an example image or `--output-pkl` to save the modified pkl. (Currently only the the Gs network is modified)

- low_res_pkl: Path, # Pickle file from which to take low res layers
- high_res_pkl: Path, # Pickle file from which to take high res layers
- resolution: int, # Resolution level at which to switch between models
- level: int  = 0, # Switch at Conv block 0 or 1?
- blend_width: Optional[float] = None, # None = hard switch, float = smooth switch (logistic) with given width
- output_grid: Optional[Path] = "blended.jpg", # Path of image file to save example grid (None = don't save)
- seed: int = 0, # seed for random grid
- output_pkl: Optional[Path] = None, # Output path of pickle (None = don't save)
- verbose: bool = False, # Print out the exact blending fraction

         

In [ ]:
!python blend_models.py --help

Use as an example a model I fine-tuned at 256 from FFHQ to data scraped from NASA DSCOVR:EPIC satellite (https://epic.gsfc.nasa.gov/).

For the output of the fine-tuned model see the tweet below (https://twitter.com/Buntworthy/status/1295445259971899393)

I'm actually going to use a model from earlier in training than the result shown in the tweet, this makes the blending a bit nicer.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf

%cd /content/
!git clone https://github.com/NVlabs/stylegan2-ada.git
%cd stylegan2-ada

!nvcc test_nvcc.cu -o test_nvcc -run

print('Tensorflow version: {}'.format(tf.__version__) )
!nvidia-smi -L
print('GPU Identified at: {}'.format(tf.test.gpu_device_name()))

In [ ]:
!cp /content/stylegan2/blend_models.py /content/stylegan2-ada/
!cp /content/stylegan2/training/misc.py /content/stylegan2-ada/training/
!cp /content/stylegan2/grid_vid.py /content/stylegan2-ada/

In [ ]:
####
# Here you get your fine-tuned PKL and name it other.pkl

!wget 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/metfaces.pkl' -O other.pkl


In [ ]:
####
# Here you get your FFHQ PKL and name it ffhq.pkl

!wget 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl' -O ffhq.pkl

In [ ]:
from IPython.display import Image 

In [ ]:
!mkdir -p "/content/drive/My Drive/colab_data/blend"

Time to try out a bunch of different resolutions for the switch and display the results. Remember the earth model provides the low-resolution layers, i.e. the "structure", and the original faces model the high-resolution, i.e. the "texture"

I'm going to run the main function in blend_models.py in a python loop, but you can also run it from the command line, something like

`python blend_models.py epic-slim-256-000040.pkl stylegan2-ffhq.pkl 64 --output-grid "blended.jpg"`

In [ ]:
import blend_models

resolutions = (4, 8, 16, 32, 64, 128, 256, 512)
#resolutions = (16, 32, 64)
for res in resolutions:
  filename = f"/content/drive/My Drive/colab_data/blend/other-ffhq-blended-{res}.jpg"
  blend_models.main("other.pkl", "ffhq.pkl", res, output_grid=filename)
  img = Image(filename=filename)
  print(f"blending at {res}x{res}")
  display(img)

In [ ]:
import blend_models

resolutions = (4, 8, 16, 32, 64, 128, 256, 512)
#resolutions = (16, 32, 64)
for res in resolutions:
  filename = f"/content/drive/My Drive/colab_data/blend/ffhq-other-blended-{res}.jpg"
  blend_models.main("ffhq.pkl", "other.pkl", res, output_grid=filename)
  img = Image(filename=filename)
  print(f"blending at {res}x{res}")
  display(img)

In [ ]:

%cd /content/stylegan2-ada/
#Here you choose the layer for the swapping in which your fine-tuned PKL is on the "Low res" side
!python blend_models.py other.pkl ffhq.pkl 64 --output-pkl="other-ffhq-blended-64.pkl"
#Here you choose the layer for the swapping in which the FFHQ PKL is on the "Low res" side
!python blend_models.py ffhq.pkl other.pkl 8 --output-pkl="ffhq-other-blended-8.pkl"



In [ ]:
#Save a copy of your blended PKLs in your drive
!cp "other-ffhq-blended-64.pkl" "/content/drive/My Drive/colab_data/blend/MetFaces-FFHQ-blended-64.pkl"
!cp "ffhq-other-blended-8.pkl"  "/content/drive/My Drive/colab_data/blend/FFHQ-MetFaces-blended-8.pkl"

In [ ]:
#Make interpolation animations

In [ ]:
%cd /content/stylegan2-ada/
import blend_models
!python grid_vid.py ffhq-other-blended-8.pkl
!mv "output.mp4" "/content/drive/My Drive/colab_data/blend/FFHQ-MetFaces-blended-8.mp4"

In [ ]:
#%cd /content/stylegan2/
%cd /content/stylegan2-ada/
import blend_models
!python grid_vid.py "other-ffhq-blended-64.pkl"
!mv "output.mp4" "/content/drive/My Drive/colab_data/blend/MetFaces-FFHQ-blended-64.mp4"

In [ ]:
#%cd /content/stylegan2/
%cd /content/stylegan2-ada/
import blend_models
!python grid_vid.py other.pkl
!mv "output.mp4" "/content/drive/My Drive/colab_data/blend/MetFaces.mp4"

In [ ]:
#%cd /content/stylegan2/
%cd /content/stylegan2-ada/
import blend_models
!python grid_vid.py ffhq.pkl
!mv "output.mp4" "/content/drive/My Drive/colab_data/blend/FFHQ.mp4"